# Dependencies

In [1]:
import requests
import logging
import boto3
from botocore.exceptions import ClientError
from base64 import b64encode
import pprint as pp
from xml.etree import ElementTree as ET

# API credentials
import api_config

# Vimeo setup
import vimeo

v = vimeo.VimeoClient(
    token=api_config.vimToken,
    key=api_config.vimClient,
    secret=api_config.vimSecret
)

C:\Users\mahlo\AppData\Roaming\Python\Python36\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (2.0.3) doesn't match a supported version!
  RequestsDependencyWarning)


# Pull the videos down from TwentyThree

In [2]:
# Define the authorization endpoints
urlRequestToken = 'http://api.visualplatform.net/oauth/request_token'
urlAccessToken = 'http://api.visualplatform.net/oauth/access_token'
urlAuthorize = 'http://api.visualplatform.net/oauth/authorize'
urlCommunicationInternal = 'http://internal.23video.com/api/method'
urlCommunicationMarketing = 'http://marketing.videomarketingplatform.co/api/method'
urlCommunicationProduct = 'http://accelo-product.videomarketingplatform.co/api/method'

In [3]:
# Define the API Credentials
ttConsumerKey = api_config.ttKey
ttConsumerSecret = api_config.ttConsumerSecret
ttToken = api_config.ttToken
ttTokenSecret = api_config.ttTokenSecret

In [30]:
# Define the request headers
headers = {
    "Content-Type": "application/json",
    "APIKey": f"{ttToken}"    
}
print(headers)

{'Content-Type': 'application/json', 'APIKey': '86846-fP3BIu4qfY4ZtC8FMrZD'}


In [5]:
# Get the count of total videos to download
response = requests.get('http://accelo-product.videomarketingplatform.co/api/photo/list', headers=headers)

targetElement = 1
iterator = 0

for i in response:
    if iterator == targetElement:
        i = str(i)
        splitEquals = i.split("=")
        splitQuote = splitEquals[1].split('\"')
        totalRecords = splitQuote[1]
        print(f"Total records: {totalRecords}")
    
    iterator += 1

Total records: 59


In [6]:
response = requests.get('http://accelo-product.videomarketingplatform.co/api/photo/list?format=json', headers=headers)

In [7]:
# Get the list of videos from 23
response = requests.get('http://accelo-product.videomarketingplatform.co/api/photo/list?size=100', headers=headers)
root = ET.fromstring(response.content)

In [27]:
# Get the videos from 23
if int(totalRecords) < 100:
    totalRecords = 100
    
vidNum = 1
page = 1
videos = {}

print("----- Begin -----")
while page <= totalRecords/100:
    
    response = requests.get(f'http://accelo-product.videomarketingplatform.co/api/photo/list?size=100&p={page}', headers=headers)
    root = ET.fromstring(response.content)

    # Iterate over the XML response, and create a new object containing just the desired fields
    
    
    vidCount = len(root.getchildren())


    for child in root.iter('*'):
        # Clear the variables for this iteration
        ownerRaw = ''
        category = ''
        title = ''
        description = ''
        link = ''


        if child.tag == 'photo':
            print("---------------------------------------")
            print(f"Reviewing video #{vidNum} of {vidCount}")

            try:
                title = child.attrib['title']
                print(f"Title: {title}")
                
                description = child.attrib['title']
                print(f"Description: {description}")
            except:
                print("No title")
                print("No description")

            try:
                ownerRaw = child.attrib['username']
                print(f"Owner: {ownerRaw}")
            except:
                print("No User URL")

            if child.attrib['video_1080p_download'] != '':
                link = child.attrib['video_1080p_download']
                print(f"Download link: {link}")

            try:
                category = child.attrib['album_title']
                print(f"Category: {category}")
            except:
                print("No category")


            # Add the video information to the videos object
            videos[f'video_{vidNum}'] = {
                'Owner': ownerRaw,
                'Category': category,
                'Title': title,
                'Description': description,
                'Link': link
            }

            # Increment the video counter
            vidNum+=1


        elif child.tag == 'content_text':
            try:
                description = child.attrib['content_text']
                print(f"Description: {description}")
            except:
                continue

        else:
            continue


    page += 1
    print("---------------------------------------") 
    
print("----- Complete -----")

----- Begin -----
---------------------------------------
Reviewing video #1 of 59
Title: Faces & Places | Tasneem
Description: Faces & Places | Tasneem
Owner: andrewgantan
Download link: /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #2 of 59
Title: Molly Flynn - Faces of Accelo.mov
Description: Molly Flynn - Faces of Accelo.mov
Owner: andrewgantan
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #3 of 59
Title: Faces & Places | Anthony
Description: Faces & Places | Anthony
Owner: andrewgantan
Download link: /49543315/56299210/29fbae46168d1c5e85cd15900a23a257/video_1080p/faces-places-anthony-video.mp4
Category: Faces & Places of Accelo
---------------------------------------
Reviewing video #4 of 59
Title: Faces & Places | Pat
Description: Faces & Places | Pat
Owner: andrewgantan
Download link: /49543318/56

In [19]:
# Create a list of unique video uploaders so that they can be mapped, and Categories to be created

def getUnique(videos):
    print("Getting unique data")

    uniqueOwners = []
    uniqueCategories = []

    for key, value in videos.items():
        #print(f"Key: {key}, Value: {value}")
        for key, value in value.items():
            if key == 'Owner':
                if value not in uniqueOwners:
                    uniqueOwners.append(value)

            if key == 'Category':
                if value not in uniqueCategories:
                    uniqueCategories.append(value)

    print(f'List of unique owners: {uniqueOwners}')
    print(f'List of unique categories: {uniqueCategories}')
    
    return(uniqueOwners, uniqueCategories)

In [15]:
# Function to map the owner

def mapOwners(videos):
    iterator = 1
    
    for vidNum, data in videos.items():
        video = vidNum
        owner = videos[video]['Owner']
        
        print(f"Iteration #{iterator}")
        print(f"First loop: {vidNum}, {data}")
        
#         print(f"Reviewing video # {video}")
#         print(f"Current owner: {owner}")
#         print(f"{vidNum} & {data}")

        
        for key, value in data.items():
            
            print(f"Second loop: {key}: {value}")
#             print("Now check owner \n")
            if key == 'Owner':
                if value == 'andrewgantan':
                    owner = 'Andrew Gantan'
                    print(f"New owner: {owner}")
                elif value == 'christaballingit':
                    owner = 'Christa Ballingit'
                    print(f"New owner: {owner}")
                elif value == 'coreyayers':
                    owner = 'Corey Ayers'
                    print(f"New owner: {owner}")
                elif value == 'alyssaivancevich':
                    owner = 'Alyssa Ivancevich'
                    print(f"New owner: {owner}")
                elif value == ('serenachan1' or 'serenachan'):
                    owner = 'Serena Chan'
                    print(f"New owner: {owner}")
                elif value =='peterescartin':
                    owner = 'Peter Escartin'
                    print(f"New owner: {owner}")
                
                    
                    
                else:
                    print("Owner not changed")
                    break
                    
                print("Iteration over")
                
        iterator += 1
                    
                    

    print(videos)
    
    return(videos)

In [21]:
uniqueOwners, uniqueCategories = getUnique(videos)
print(uniqueOwners)
print(uniqueCategories)

Getting unique data
List of unique owners: ['andrewgantan', 'christaballingit', 'coreyayers', 'alyssaivancevich', 'shunleisin', 'serenachan1', 'peterescartin', 'serenachan']
List of unique categories: ['Faces & Places of Accelo', 'Accelo Best Practices', 'Webinars', 'Accelo Help Videos', 'New User Videos: Intro to Accelo']
['andrewgantan', 'christaballingit', 'coreyayers', 'alyssaivancevich', 'shunleisin', 'serenachan1', 'peterescartin', 'serenachan']
['Faces & Places of Accelo', 'Accelo Best Practices', 'Webinars', 'Accelo Help Videos', 'New User Videos: Intro to Accelo']


In [23]:
videos = mapOwners(videos)
print(videos)

Iteration #1
First loop: video_1, {'Owner': 'andrewgantan', 'Category': 'Faces & Places of Accelo', 'Title': 'Faces & Places | Tasneem', 'Description': 'Faces & Places | Tasneem', 'Link': '/49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4'}
Second loop: Owner, andrewgantan
New owner: Andrew Gantan
Iteration over
Second loop: Category, Faces & Places of Accelo
Second loop: Title, Faces & Places | Tasneem
Second loop: Description, Faces & Places | Tasneem
Second loop: Link, /49543318/56300288/95eec35b4ddf80d9c3bb148d9581bcc9/video_1080p/faces-places-tasneem-video.mp4
Iteration #2
First loop: video_2, {'Owner': 'andrewgantan', 'Category': 'Faces & Places of Accelo', 'Title': 'Molly Flynn - Faces of Accelo.mov', 'Description': 'Molly Flynn - Faces of Accelo.mov', 'Link': '/49543322/57216833/75b2ac70a611838e4181d257e3430e2b/video_1080p/molly-flynn-faces-of-accelomov-video.mp4'}
Second loop: Owner, andrewgantan
New owner: Andrew Gantan
Iteration o

In [31]:
response = requests.get('http://accelo-product.videomarketingplatform.co/api/photo/list?size=100&p=1', headers=headers)
print(response.text)

<?xml version="1.0" encoding="UTF-8"?>
<response status="ok" permission_level="anonymous" p="1" offset="0" size="100" total_count="59" cached="1" cache_time="1590181337">
    <photo photo_id="56300288" title="Faces &amp; Places | Tasneem" tree_id="49543318" token="95eec35b4ddf80d9c3bb148d9581bcc9" promoted_p="0" protected_p="0" protection_method="" album_id="57497551" album_title="Faces &amp; Places of Accelo" album_hide_p="0" all_albums="57497551" published_p="1" one="/faces-places-tasneem" publish_date_ansi="2019-11-14 13:45:00" publish_date_epoch="1573739100" publish_date__date="November 14, 2019" publish_date__time="01:45 PM" creation_date_ansi="2019-10-02 15:19:19" creation_date_epoch="1570029559" creation_date__date="October 02, 2019" creation_date__time="03:19 PM" original_date_ansi="2019-10-02 22:19:19" original_date__date="October 02, 2019" original_date__time="10:19 PM" view_count="63" avg_playtime="56.5529651877592" number_of_comments="0" number_of_albums="1" number_of_tags=

In [ ]:
root = ET.fromstring(response.content)

for child in root.iter('*'):
    print(child.tag)
    print(child.attrib)

# Vimeo

## Test the API Connection

In [ ]:
# Make the request to the server for the "/me" endpoint.
about_me = v.get('/me')

# Make sure we got back a successful response.
assert about_me.status_code == 200

# Load the body's JSON data.
print(about_me.json())

In [ ]:
videos = {"vid1": 
          {"title": "The Title",
           "description": "A cool video",
           "Download Link": "google.com"
          },
          {}

In [ ]:
# Perform the upload
file_name = '{path_to_a_video_on_the_file_system}'
uri = client.upload(file_name, data={
  'name': 'Untitled',
  'description': 'The description goes here.'
})

print 'Your video URI is: %s' % (uri)